In [ ]:
import json
from datetime import datetime

# File to store employee data
EMPLOYEE_DB = "employees.json"

# Employee class
class Employee:
    def __init__(self, emp_id, name, department, salary):
        self.emp_id = emp_id
        self.name = name
        self.department = department
        self.salary = salary
        self.performance_reviews = []

    def add_performance_review(self, review):
        self.performance_reviews.append(review)

    def update_salary(self, new_salary):
        self.salary = new_salary

    def promote(self, new_department):
        self.department = new_department

    def to_dict(self):
        return {
            "emp_id": self.emp_id,
            "name": self.name,
            "department": self.department,
            "salary": self.salary,
            "performance_reviews": [review.to_dict() for review in self.performance_reviews]
        }

# Department class
class Department:
    def __init__(self, department_name):
        self.department_name = department_name
        self.employees = []

    def add_employee(self, employee):
        self.employees.append(employee)

    def get_performance_summary(self):
        performance = {}
        for emp in self.employees:
            for review in emp.performance_reviews:
                if review.rating not in performance:
                    performance[review.rating] = 1
                else:
                    performance[review.rating] += 1
        return performance

# PerformanceReview class
class PerformanceReview:
    def __init__(self, review_date, rating, comments):
        self.review_date = review_date
        self.rating = rating
        self.comments = comments

    def to_dict(self):
        return {
            "review_date": self.review_date,
            "rating": self.rating,
            "comments": self.comments
        }

# Helper functions for CRUD operations
def load_employees():
    try:
        with open(EMPLOYEE_DB, 'r') as f:
            data = json.load(f)
            employees = []
            for emp_data in data:
                emp = Employee(emp_data['emp_id'], emp_data['name'], emp_data['department'], emp_data['salary'])
                for review_data in emp_data['performance_reviews']:
                    review = PerformanceReview(review_data['review_date'], review_data['rating'], review_data['comments'])
                    emp.add_performance_review(review)
                employees.append(emp)
            return employees
    except FileNotFoundError:
        return []

def save_employees(employees):
    with open(EMPLOYEE_DB, 'w') as f:
        json.dump([emp.to_dict() for emp in employees], f)

def create_employee(emp_id, name, department, salary):
    employees = load_employees()
    new_employee = Employee(emp_id, name, department, salary)
    employees.append(new_employee)
    save_employees(employees)

def get_employee_by_id(emp_id):
    employees = load_employees()
    for emp in employees:
        if emp.emp_id == emp_id:
            return emp
    return None

def get_employee_by_name(name):
    employees = load_employees()
    return [emp for emp in employees if name.lower() in emp.name.lower()]

def get_employee_by_department(department):
    employees = load_employees()
    return [emp for emp in employees if department.lower() in emp.department.lower()]

def update_employee_salary(emp_id, new_salary):
    employees = load_employees()
    emp = get_employee_by_id(emp_id)
    if emp:
        emp.update_salary(new_salary)
        save_employees(employees)
        return True
    return False

def add_performance_review(emp_id, review_date, rating, comments):
    employees = load_employees()
    emp = get_employee_by_id(emp_id)
    if emp:
        review = PerformanceReview(review_date, rating, comments)
        emp.add_performance_review(review)
        save_employees(employees)
        return True
    return False

def promote_employee(emp_id, new_department):
    employees = load_employees()
    emp = get_employee_by_id(emp_id)
    if emp:
        emp.promote(new_department)
        save_employees(employees)
        return True
    return False

def generate_department_summary(department_name):
    employees = load_employees()
    department = Department(department_name)
    for emp in employees:
        if emp.department.lower() == department_name.lower():
            department.add_employee(emp)
    return department.get_performance_summary()

# Example Usage

# Create employees
create_employee(1, "Alice", "Sales", 50000)
create_employee(2, "Bob", "Engineering", 70000)

# Add performance reviews
add_performance_review(1, str(datetime.now()), 5, "Excellent performance")
add_performance_review(2, str(datetime.now()), 4, "Good work")

# Search by ID, Name, or Department
print(get_employee_by_id(1))
print(get_employee_by_name("Alice"))
print(get_employee_by_department("Sales"))

# Update salary and promote employee
update_employee_salary(1, 55000)
promote_employee(1, "Management")

# Generate department performance summary
print(generate_department_summary("Sales"))
